In [2]:
#Import libraries
import ee
import geemap.eefolium as geemap

# Authentication and Initialisation

The earth engine API has to be authenticated when used for the first time. 

In [2]:
# Trigger the authentication flow. Uncomment if needed
#ee.Authenticate()

Enter verification code: 4/1AY0e-g6mZPjK2v9jdPR9S_oWvr1TyGzSgCUpGmtmiNB3GGK2lJ12T-0YhKI

Successfully saved authorization token.


In [3]:
#Initialise earth engine
ee.Initialize()

In [4]:
#Time period
startTime = '2019-12-01'
endTime = '2020-02-28' 

In [5]:
# Define and visualise the AOI
geometry = ee.Geometry.Polygon([[-5.889693732515384,9.061340118650035],[2.572525505765846,9.061340118650035],[2.572525505765846,15.066746580273666], [-5.889693732515384,15.066746580273666], [-5.889693732515384,9.061340118650035]])
Map = geemap.Map(center=[9,3], zoom=4)
Map.addLayer(geometry)
Map

# Cloud Masks

In [6]:
# Sentinel 2 cloud mask

def maskS2clouds(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

In [79]:
#Landsat 8 cloud mask
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
    qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
            qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# Indices

In [80]:
#Modified soil adjusted vegetation Index 
#Sentinel 2
def msavi (image):
    addMSAVI = image.expression(
    '(((2*NIR +1)- (sqrt((2*NIR+1)**2-(8*( NIR - RED)))))/2)', {
        'NIR': image.select('B8'),
        'RED': image.select('B4').float()
        })
    return image.addBands(addMSAVI.rename('msavi'))

#Landsat 8
def msavi_l8(image):
    addMSAVI_l8 = image.expression(
    '(((2*NIR +1)- (sqrt((2*NIR+1)**2-(8*( NIR - RED)))))/2)', {
        'NIR': image.select('B5'),
        'RED': image.select('B4').float()
        })
    return image.addBands(addMSAVI_l8.rename('msavi_l8'))

In [81]:
#Normalized DIffernce Vegetation Index
# Sentinel 2
def ndvi(image):
    addNDVI = image.normalizedDifference(['B8', 'B4'])
    return image.addBands(addNDVI.rename('ndvi'))


# landsat 8
def ndvi_l8(image):
    addNDVI_l8 = image.normalizedDifference(['B5', 'B4'])
    return image.addBands(addNDVI_l8.rename('ndvi_l8'))


In [56]:
#Normalized difference water index
def ndwi(image):
    addNDWI = image.normalizedDifference(['B3', 'B8'])
    return image.addBands(addNDWI.rename('ndwi'))


In [82]:
#Enhanced vegetation index 
#Landsat 8
def evi_l8(image):
    addEVI_l8 = image.expression(
    '(2.5*((NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
        'NIR': image.select('B5'),
        'BLUE': image.select('B2'),
        'RED': image.select('B4').float()
        })
    return image.addBands(addEVI_l8.rename('evi_l8'))
#S2
def evi_s2(image):
    addEVI_s2 = image.expression(
    '2.5*(NIR-RED)/((NIR+6*RED-7.5*BLUE)+1)', {
        'NIR': image.select('B8'),
        'BLUE': image.select('B2'),
        'RED': image.select('B4').float()
        })
    return image.addBands(addEVI_s2.rename('evi_s2'))

In [12]:
#Normalised Difference Water Index
#S2
def ndmi(image):
    addNDVI = image.normalizedDifference(['B8A', 'B11'])
    return image.addBands(addNDVI.rename('ndmi'))

# Satellite data

All satellite scenes that intersect with the given search parameters (AOI, time range ..) are stored in an image collection. The images can be filtered by cloud cover as well.

## Sentinel-2 MSI

In [13]:
# Sentinel 2 MSI surface reflectance - image collection
#filter by area and time, apply functions to each image in the collection
collection = (ee.ImageCollection('COPERNICUS/S2_SR') 
 .filterDate(startTime, endTime) 
 .filterBounds(geometry) 
 .map(maskS2clouds) 
 .map(msavi) 
 .map(ndwi) 
 .map(ndvi)
 .map(ndmi)
 .map(evi_s2))

#Visualise the collection
rgbVis = {
  'min': 0.0,
  'max': 0.4,
  'bands': ['B4', 'B8', 'B2'],
  'gamma': [0.95, 1.1, 1]
};

#S2 projection is used for the final projection of the landcover map
S2_SR_med = collection.median()
S2_proj = S2_SR_med.projection()

Map.addLayer(collection, rgbVis, 'Sentinel 2')
Map

## Landsat 8 OLI/TIRS Tier 1

In [83]:
# Landsat 8 image collection
landsat8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
                  .filterDate(startTime, endTime)
                  .filterBounds(geometry)
                  .map(maskL8sr)
                  .map(ndvi_l8)
                  .map(msavi_l8)
                  .map(evi_l8))
#print(landsat8.size().getInfo())

## Landsat 8 land surface temperature

The following code provides a land surface temperature product based on landsat 8 input data. Uncomment visualisation options as needed.

In [84]:
#median
image = landsat8.median()
#print(image, 'image');
#Map.addLayer(image, vizParams2)

# NDVI:
ndvi = image.normalizedDifference(['B5', 'B4']) \
.rename('NDVI')
#ndviParams = {min: -1, max: 1, palette: ['blue', 'white', 'green']}
#print(ndvi,'ndvi')
#Map.addLayer(ndvi, ndviParams, 'ndvi')

#

#{
# find the min and max of NDVI
Min= ee.Number(ndvi.reduceRegion(**{
 'reducer': ee.Reducer.min(),
 'geometry': geometry,
 'scale': 30,
 'maxPixels': 1e9
}).values().get(0))
#print(min, 'min');
Max= ee.Number(ndvi.reduceRegion(**{
   'reducer': ee.Reducer.max(),
   'geometry': geometry,
   'scale': 30,
   'maxPixels': 1e9
   }).values().get(0))
#print(Max, 'max')
#}    

#fractional vegetation
fv = ndvi.subtract(Min)  \
.divide(Max\
        .subtract(Min)) \
.rename('FV')
#print(fv, 'fv')
#Map.addLayer(fv)

######/


  #Emissivity

a= ee.Number(0.004)
b= ee.Number(0.986)
EM=fv.multiply(a) \
.add(b) \
.rename('EMM')
  
""""  {
 min = ee.Number(EM.reduceRegion({
   reducer: ee.Reducer.min(),
   geometry: geometry,
   scale: 30,
   maxPixels: 1e9
   }).values().get(0))
print(min, 'min');
 max = ee.Number(EM.reduceRegion({
    reducer: ee.Reducer.max(),
   geometry: geometry,
   scale: 30,
   maxPixels: 1e9
   }).values().get(0))
print(max, 'max')
}"""
  # imageVisParam2 = {min: 0.98, max: 0.99, palette: ['blue', 'white', 'green']}
  #Map.addLayer(EM, imageVisParam2,'EMM')




'"  {\n min = ee.Number(EM.reduceRegion({\n   reducer: ee.Reducer.min(),\n   geometry: geometry,\n   scale: 30,\n   maxPixels: 1e9\n   }).values().get(0));\nprint(min, \'min\');\n max = ee.Number(EM.reduceRegion({\n    reducer: ee.Reducer.max(),\n   geometry: geometry,\n   scale: 30,\n   maxPixels: 1e9\n   }).values().get(0));\nprint(max, \'max\')\n}'

In [85]:
#select thermal band 10(with brightness tempereature), no BT calculation 
thermal= image.select('B10').multiply(0.1)
 
{
   Min : ee.Number(thermal.reduceRegion(**{
    'reducer': ee.Reducer.min(),
   'geometry': geometry,
   'scale': 30,
   'maxPixels': 1e9
   }) \
   .values() \
   .get(0)),
#print(min, 'min')
  Max : ee.Number(thermal.reduceRegion(**{
    'reducer': ee.Reducer.max(),
   'geometry': geometry,
   'scale': 30,
   'maxPixels': 1e9
   }).values().get(0))
#print(max, 'max')
}
#b10Params = {min: 289, max: 318, palette: ['blue', 'white', 'green']}
# Map.addLayer(thermal, b10Params, 'thermal')



{<ee.ee_number.Number at 0x1aa3da5db80>: <ee.ee_number.Number at 0x1aa3da5b1f0>,
 <ee.ee_number.Number at 0x1aa3da5dca0>: <ee.ee_number.Number at 0x1aa3da5bf40>}

In [86]:
LST = thermal.expression(
    '(Tb/(1 + (0.001145* (Tb / 1.438))*log(Ep)))-273.15', {
      'Tb': thermal.select('B10'),
      'Ep': EM.select('EMM')
})\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })

  #LST c,d,f, p1, p2, p3 are assigned iables to write equaton easily
"""""   c= ee.Number(1)
   d= ee.Number(0.00115)
   f= ee.Number(1.4388)


 p1= ee.Number(thermal.multiply(d).divide(f))
 p2= ee.Number(Math.log(EM))
 p3= ee.Number(p1.multiply(p2).add(c))



 LST= (thermal.divide(p3)).rename('LST');

 LSTimage = ee.Image(LST)"""""

{
 Min : ee.Number(LST.reduceRegion(**{
   'reducer': ee.Reducer.min(),
   'geometry': geometry,
   'scale': 30,
   'maxPixels': 1e9
   }).values().get(0)),
#print(min, 'min');
 Max : ee.Number(LST.reduceRegion(**{
   'reducer': ee.Reducer.max(),
   'geometry': geometry,
   'scale': 30,
   'maxPixels': 1e9
   }).values().get(0))
#print(max, 'max')
}

"""LSTVis = {
  min:17,
  max: 46,
  palette: ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'],
}"""

#Map.addLayer(LST, LSTVis,'LST')

"LSTVis = {\n  min:17,\n  max: 46,\n  palette: ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'],\n}"

## Sentinel 1  GRD data

Sentinel 1 GRD data is filtered by AOI, time range, polarisation, instrument mode and orbit properties. Only VV and VH polarisations are available for the given AOI. A speckle filter is apllied to all sentinel 1 image collections.

In [18]:
# Speckle filter
def filterSpeckles (img):
    vv = img.select('VV') #select the VV polarization band
    vv_smoothed = vv.focal_median(30,'circle','meters') \
    .rename('VV_Filtered') #Apply a focal median filter
    vh = img.select('VH') #select the VV polarization band
    vh_smoothed = vh.focal_median(30,'circle','meters')\
    .rename('VH_Filtered') #Apply a focal median filter
    return img.addBands(vv_smoothed) \
           .addBands(vh_smoothed) # Add filtered VV band to original image

The first S1 image collection covers three months for each season, where either no rain would be expected (dry season) or the rainy season is ongoing in northern Ghana and northern Burkina Faso.

In [19]:
#Sentinel 1, season specific. Change date accordingly.
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    #.filter(ee.Filter.eq('resolution_meters', '10'))
    .filterDate('2019-12-01', '2020-02-28')
    .filterBounds(geometry)
    .select('V.')
    .map(filterSpeckles))
#print(sentinel1.size().getInfo())

Annual S1 data is used for the dry season and the wet season map.

In [20]:
#Sentinel 1, annual data 2018
sentinel1_2018 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    #.filter(ee.Filter.eq('resolution_meters', '10'))
    .filterDate('2018-01-01', '2018-12-30')
    .filterBounds(geometry)
    .map(filterSpeckles))
#print(sentinel1_2018.size().getInfo())

In [21]:
#Sentinel 1, annual data 2019
sentinel1_2019 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    #.filter(ee.Filter.eq('resolution_meters', '10'))
    .filterDate('2019-01-01', '2019-12-30')
    .filterBounds(geometry)
    .map(filterSpeckles))
#print(sentinel1_2019.size().getInfo())

In [22]:
#Sentinel 1, annual data 2020
sentinel1_2020 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    #.filter(ee.Filter.eq('resolution_meters', '10'))
    .filterDate('2020-01-01', '2020-12-31')
    .filterBounds(geometry)
    .map(filterSpeckles))
#print(sentinel1_2020.size().getInfo())

Monthly S1 data. The number of image collections can depend on the season and area. The following are specific to the dry season in the semi-arid Ghana/Burkina Faso.

In [23]:
#S1 monthly data
sentinel1_dec =  (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate('2019-12-01', '2019-12-31')
    .filterBounds(geometry)
    .select('V.')
    .map(filterSpeckles))
#print(sentinel1_dec.size().getInfo())

In [24]:
sentinel1_jan = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate('2020-01-01', '2020-01-31')
    .filterBounds(geometry)
    .select('V.')
    .map(filterSpeckles))

#print(sentinel1_jan.size().getInfo())

In [25]:
sentinel1_feb = (ee.ImageCollection('COPERNICUS/S1_GRD')
  # Filter to get images with VV and VH dual polarization.
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  .filter(ee.Filter.eq('instrumentMode', 'IW'))
  .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
  .filterDate('2020-02-01', '2020-02-28')
  .filterBounds(geometry)
  .select('V.')
  .map(filterSpeckles))

print(sentinel1_feb.size().getInfo())

71


In [26]:
sentinel1_mar = (ee.ImageCollection('COPERNICUS/S1_GRD')
  # Filter to get images with VV and VH dual polarization.
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  .filter(ee.Filter.eq('instrumentMode', 'IW'))
  .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
  .filterDate('2020-03-01', '2020-03-31')
  .filterBounds(geometry)
  .select('V.')
  .map(filterSpeckles))
print(sentinel1_mar.size().getInfo())

82


In [27]:
sentinel1_apr = (ee.ImageCollection('COPERNICUS/S1_GRD')
    # Filter to get images with VV and VH dual polarization.
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate('2020-04-01', '2020-04-30')
    .filterBounds(geometry)
    .select('V.')
    .map(filterSpeckles))

print(sentinel1_apr.size().getInfo())

78


## ALOS PALSAR

In [28]:
#Is image
alos_data = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')
                  .filter(ee.Filter.date('2017-01-01', '2018-01-01'))
                  .select('HV'))

## Digital Elevation Model

In [29]:
#Is image collection and has to be mosaiced to return a single image
dem = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').filterBounds(geometry).mosaic()

# Temporal reduction

The reducers used depend on the season. A maxmimum reducer may not be useful in the wet season as cloud cover returns high reflectance values.

In [41]:
reducer = ee.Reducer.median().combine(
    reducer2= ee.Reducer.max(), sharedInputs =True)
reducer = reducer.combine(
    reducer2= ee.Reducer.min(), sharedInputs =True)
reducer = reducer.combine(
    reducer2 = ee.Reducer.variance(),sharedInputs =True)
reducer = reducer.combine(
    reducer2= ee.Reducer.mean(), sharedInputs=True)

In [87]:
composite_s1 = sentinel1.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s12018 = sentinel1_2018.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_2019 = sentinel1_2019.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_2020 = sentinel1_2020.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_dec = sentinel1_dec.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_jan = sentinel1_jan.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_feb = sentinel1_feb.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_s1_apr = sentinel1_apr.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
s1_change_comp = composite_s1_apr.subtract(composite_s1_jan)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite = collection.reduce(reducer)
composite_l8 = landsat8.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })
composite_alos = alos_data.reduce(reducer)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })

# Input image

In [88]:
composite = composite.select("B.*", "msavi_.*", "ndvi_.*", "ndwi_.*", "ndmi_.*") \
                  .addBands(composite_s1) \
                  .addBands(composite_s1_2019) \
                  .addBands(composite_s1_2020) \
                  .addBands(composite_s1_dec) \
                  .addBands(composite_s1_jan) \
                  .addBands(composite_s1_feb) \
                  .addBands(s1_change_comp) \
                  .addBands(composite_l8.select("B.*", "msavi_.*", "ndvi_.*", "evi_.*")) \
                  .addBands(LST) \
                  .addBands(composite_alos) \
                  .addBands(composite_s12018) \
                  .addBands(dem.select('DSM'))
#print(composite.bandNames().getInfo())

# Reference data

If the refernce data is an asset you can access it by calling a feature collection with the path to your asset.

In [44]:
data = ee.FeatureCollection('users/username/folder/file') \
  .merge(ee.FeatureCollection('users/username/folder/file')) \
  .merge(ee.FeatureCollection('users/username/folder/file')) \
  .merge(ee.FeatureCollection('users/username/folder/file')) 

sample = data.randomColumn()
print(sample.size().getInfo())

3820


Split refernce data into training and testing data. The number used for splitting the data may adjusted to whichever fits best.

In [45]:
split = 0.7  # Roughly 70% training, 30% testing.
newfc = sample.filter(ee.Filter.lt('random', split))
validation_fc = sample.filter(ee.Filter.gte('random', split))

Printing the list of bands or the number of bands executes part of the code before the classifier is trained and accuracies are estimated. See the GEE guide for further information on defered execution.

In [89]:
band_list = ee.List(composite.bandNames())
print(band_list.size().getInfo())

322


In [36]:
#Uncomment if band values for the reference data is needed
'''training_stats = composite.sampleRegions(
    collection= data,
    properties= ['class_new'],
    scale= 20,
    tileScale= 4
  )'''

"training_stats = composite.sampleRegions(\n    collection= data,\n    properties= ['class_new'],\n    scale= 20,\n    tileScale= 4\n  )"

Sample the band values for training and testing the classifier. The output mode of the classifier can be changed to 'MULTIPORBABILITY' to get the probability estimation for each pixel and class.

In [40]:
#Sample band values for training data
training = composite.sampleRegions(
    collection= newfc,
    properties= ['class_new'],
    scale= 20,
    tileScale= 4
  )

# train the classifier with the training data and input bands 
classifier = ee.Classifier.smileRandomForest(200)  \
.train(
    features= training,
    classProperty = 'class_new',
    inputProperties= composite.bandNames()   
)

#Sample band values for the testing data
validation = composite.sampleRegions(
    collection= validation_fc,
    properties= ['class_new'],
    scale= 20,
    tileScale= 16
)

# Classify the testing data  
validated = validation.classify(classifier, 'classification')

#Get the accuracy metrics for the testing data
testAccuracy = validated.errorMatrix('class_new', 'classification')
classifier_accuracy = testAccuracy.accuracy()

In [41]:
#Print accuracy metrics
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', classifier_accuracy.getInfo())
print('Kappa index:', testAccuracy.kappa().getInfo())
print('PA:', testAccuracy.producersAccuracy().getInfo())
print('UA:', testAccuracy.consumersAccuracy().getInfo())

Validation error matrix:  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 131, 9, 0, 4, 24, 14, 5], [0, 11, 160, 0, 5, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 10, 3, 0, 206, 5, 33, 0], [0, 13, 0, 0, 17, 157, 9, 3], [0, 8, 3, 0, 38, 9, 146, 0], [0, 4, 0, 0, 2, 1, 2, 111]]
Validation overall accuracy:  0.7963286713286714
Kappa index: 0.7531146408264091
PA: [[0], [0.7005347593582888], [0.903954802259887], [0], [0.8015564202334631], [0.7889447236180904], [0.7156862745098039], [0.925]]
UA: [[0, 0.7401129943502824, 0.9142857142857143, 0, 0.7573529411764706, 0.7969543147208121, 0.7156862745098039, 0.9327731092436975]]


The masked input image has to be unmasked when the classifier is applied to it, as the classifier does not classify masked pixel. Masked pixel will return empty pixels.

In [48]:
classification_result = composite.unmask().classify(classifier) \
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })'''

".reproject(**{\n      'crs': S2_proj.crs(),\n      'scale':20\n        })"

Uncomment the following to get an image from the resulting array, where each band contains the probabiltiy of a given pixel belonging to that specific class (band 1: probability of a pixel belonging to class 1/bare soil and crop residue).

In [47]:
# If output mode is multiprobability.
'''classification_resultFlat = classification_result.arrayProject([0]) \
  .arrayFlatten([['1', '2', '3', '4','5','6', '7']] )'''

In [49]:
# Color palette for visualising the map
palette = [
  '8b5402', # soil(1)e3990d   # lightbrown
  '3e3e3e', # built(2)        # darkgrey
  'b61515', # shrubs(3)       # red
  '1f8901', # tree(4)         # darkgreen
  '00ff78', # irr(5)          # lightgreen 00ff78
  '79d402', # green_shrub (6) #turquoise ccff00 91af40 79d402
  '00ccff'  # water(7)        # lightblue'91af40', #green_shrub (6)#turquoise
]

In [53]:
#Map.setCenter(-0.6318328568074372,11.497742943317558, 10)
Map2 = geemap.Map(center=[9,3], zoom=4)
Map2.addLayer(geometry)
Map2.addLayer(classification_result, {'min':1, 'max':7, 'palette': palette}, "Landcover classification")
Map2.addLayerControl()

Map2

# Importance estimates

In [42]:
#get importance of variables used to trian the RF classifier
Dict = classifier.explain()
variable_importance = ee.Dictionary(ee.Dictionary(Dict)\
.get('importance'))

#Write hte importance from dictionary to list
def importance(k, v):
    return ee.Feature(None,{'key': k,
      'value': v}) 

        
best=variable_importance.map(importance) \
     .values()
    
best_fc = ee.FeatureCollection(best)

In [44]:
#function for getting the actual value of importance
def getValues(feature):
    value_new = ee.Number(feature.get('value'))
    return feature.set('value_new', value_new)

In [45]:
#use the importance value matched with the input bandss to sort the input bands and limit to the X (here: 120) most important bands
best_var = best_fc.map(getValues) \
.sort('value_new', False) \
.limit(120) \
.reduceColumns(ee.Reducer.toList(), ['key']) \
.get('list')

#print(best.size().getInfo())
print(best_var.getInfo())

['ndwi_min', 'ndwi_median', 'B11_max_1', 'ndwi_mean', 'ndvi_max', 'VH_Filtered_variance_1', 'VH_Filtered_variance_7', 'VH_variance_7', 'ndvi_variance', 'msavi_max', 'msavi_variance', 'B11_variance', 'VH_median_2', 'VH_variance_1', 'B2_min', 'ndmi_max', 'ndvi_mean', 'B10', 'ndmi_min', 'B8_variance', 'B2_mean', 'B10_median', 'msavi_mean', 'B11_variance_1', 'B12_min', 'VH_min_2', 'ndmi_median', 'DSM', 'VH_Filtered_min_1', 'B3_min', 'angle_min_1', 'VV_Filtered_variance', 'ndvi_min', 'angle_median_1', 'B7_max', 'VH_mean_7', 'angle_min_2', 'VH_mean_1', 'B1_variance_1', 'ndmi_variance', 'VH_median_1', 'VV_max_5', 'VH_min_5', 'VV_Filtered_min_6', 'B11_mean', 'VH_Filtered_mean_7', 'ndvi_median', 'VH_Filtered_min_7', 'VV_Filtered_median_3', 'B11_min', 'angle_variance_2', 'B10_max', 'B10_variance', 'B1_variance', 'VH_Filtered_min_2', 'VH_Filtered_min_6', 'B5_variance', 'B4_mean', 'VV_median_1', 'B4_max_1', 'VH_max_1', 'angle_variance_1', 'B6_variance_1', 'VH_Filtered_variance_2', 'B11_mean_1', 'V

# Classfication with important variables

In [127]:
#Train classifier with lsit of important variables
second_classifier = ee.Classifier.smileRandomForest(250)\
.train(
      features= training,
      classProperty= 'class_new',
      inputProperties= best_var)

In [128]:
validated2 = validation.classify(second_classifier, 'classification')
# Get a confusion matrix representing expected accuracy.
testAccuracy2 = validated2.errorMatrix('class_new', 'classification')
classifier_accuracy2 = testAccuracy2.accuracy()

In [129]:
print('Validation error matrix: ', testAccuracy2.getInfo())
print('Validation overall accuracy: ', classifier_accuracy2.getInfo())
print('Kappa index:', testAccuracy2.kappa().getInfo())
print('PA:', testAccuracy2.producersAccuracy().getInfo())
print('UA:', testAccuracy2.consumersAccuracy().getInfo())

Validation error matrix:  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 137, 4, 0, 6, 23, 12, 5], [0, 12, 158, 0, 6, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 9, 4, 0, 207, 7, 30, 0], [0, 13, 0, 0, 16, 159, 7, 4], [0, 9, 2, 0, 38, 8, 147, 0], [0, 3, 0, 0, 2, 2, 2, 111]]
Validation overall accuracy:  0.8033216783216783


In [130]:
#export the error matrix to your local computer
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
testing_csv = os.path.join(out_dir, 'test_accuracy_ds.csv')
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testAccuracy2.getInfo())

In [46]:
#Classify the image. Unmask the image to avoid empty pixels
classification_result2 = composite.unmask().classify(second_classifier)\
.reproject(**{
      'crs': S2_proj.crs(),
      'scale':20
        })

In [204]:
#Visualise the map
Map.setCenter(-0.6318328568074372,11.497742943317558, 10)
Map.addLayer(classification_result2, {'min':1, 'max':7, 'palette': palette}, "Landcover classification 2")
Map.addLayerControl()

Map

The following cell exports the classification as an image. The larger the file, the longer the export takes. The cell, however only runs for a few minutes.

In [49]:
#Export the Map to google drive
task = ee.batch.Export.image.toDrive(image=classification_result2,                       # the map
                                    description= 'lulc_dryseason_v20210913_c2',          # the file name
                                    folder='lcc_image',                                  # the google drive folder
                                    region=geometry,                                     # the area the map is suppose to be exported for
                                    scale=20,                                            # the spatial resolution    
                                    maxPixels= 10000000000000,                           # the maximum number of pixels, the larger the area, the more poxiel are required
                                    fileFormat='GeoTIFF')                                # the file format




task.start()
print(task.status())

{'state': 'READY', 'description': 'lulc_dryseason_v20210913_c2', 'creation_timestamp_ms': 1631541556566, 'update_timestamp_ms': 1631541556566, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'OYPLSSKGDM6OTSTBS2E4E4NK', 'name': 'projects/earthengine-legacy/operations/OYPLSSKGDM6OTSTBS2E4E4NK'}
